In [1]:
from gates import *

In [89]:
class SRNandLatch:
    def __init__(self):
        self.q = 0
        self.not_q = 1

    def eval(self, r, s):
        if s == 0:
            self.not_q = Nand(self.q, s)
            self.q = Nand(r, self.not_q)
        elif r == 0:
            self.q = Nand(r, self.not_q)
            self.not_q = Nand(self.q, s)

        return self
    
    def __repr__(self):
        return f'q={self.q}, not_q={self.not_q}'

In [90]:
l = SRNandLatch()
l

q=0, not_q=1

In [91]:
l.eval(s=1, r=0)

q=1, not_q=0

In [92]:
l.eval(s=0, r=1)

q=0, not_q=1

In [18]:
class DFF:
    def __init__(self):
        self.q = 0
    
    def eval(self, inp):
        last_q = self.q
        self.q = inp
        return last_q


class Bit:
    def __init__(self):
        self.dff = DFF()
    
    def eval(self, inp, load):
        # Not sure how else to implement hardware feedback in code
        # This is sort of cheating, because we look at the output
        # of the DFF before running eval() on it.
        which = Mux(self.dff.q, inp, load)
        return self.dff.eval(which)

    def __str__(self):
        return str(self.dff.q)
    
    def __repr__(self):
        return str(self)


class Register:
    def __init__(self):
        self.bit0  = Bit()
        self.bit1  = Bit()
        self.bit2  = Bit()
        self.bit3  = Bit()
        self.bit4  = Bit()
        self.bit5  = Bit()
        self.bit6  = Bit()
        self.bit7  = Bit()
        self.bit8  = Bit()
        self.bit9  = Bit()
        self.bit10 = Bit()
        self.bit11 = Bit()
        self.bit12 = Bit()
        self.bit13 = Bit()
        self.bit14 = Bit()
        self.bit15 = Bit()
    
    def eval(self, inp, load):
        out0  = self.bit0.eval(inp[15], load)
        out1  = self.bit1.eval(inp[14], load)
        out2  = self.bit2.eval(inp[13], load)
        out3  = self.bit3.eval(inp[12], load)
        out4  = self.bit4.eval(inp[11], load)
        out5  = self.bit5.eval(inp[10], load)
        out6  = self.bit6.eval(inp[9],  load)
        out7  = self.bit7.eval(inp[8],  load)
        out8  = self.bit8.eval(inp[7],  load)
        out9  = self.bit9.eval(inp[6],  load)
        out10 = self.bit10.eval(inp[5], load)
        out11 = self.bit11.eval(inp[4], load)
        out12 = self.bit12.eval(inp[3], load)
        out13 = self.bit13.eval(inp[2], load)
        out14 = self.bit14.eval(inp[1], load)
        out15 = self.bit15.eval(inp[0], load)

        return out15, out14, out13, out12, out11, out10, out9, out8, \
               out7, out6, out5, out4, out3, out2, out1, out0
        
    def __str__(self):
        return f'{self.bit15} {self.bit14} {self.bit13} {self.bit12} ' + \
               f'{self.bit11} {self.bit10} {self.bit9} {self.bit8} ' + \
               f'{self.bit7} {self.bit6} {self.bit5} {self.bit4} ' + \
               f'{self.bit3} {self.bit2} {self.bit1} {self.bit0}'
    
    def __repr__(self):
        return str(self)


class RAM8:
    def __init__(self):
        self.r0 = Register()
        self.r1 = Register()
        self.r2 = Register()
        self.r3 = Register()
        self.r4 = Register()
        self.r5 = Register()
        self.r6 = Register()
        self.r7 = Register()

    def eval(self, inp, load, addr):
        ld0, ld1, ld2, ld3, ld4, ld5, ld6, ld7 = DMux8Way(load, sel=addr)

        out0 = self.r0.eval(inp, ld0)
        out1 = self.r1.eval(inp, ld1)
        out2 = self.r2.eval(inp, ld2)
        out3 = self.r3.eval(inp, ld3)
        out4 = self.r4.eval(inp, ld4)
        out5 = self.r5.eval(inp, ld5)
        out6 = self.r6.eval(inp, ld6)
        out7 = self.r7.eval(inp, ld7)

        return Mux8Way16(out0, out1, out2, out3, out4, out5, out6, out7, sel=addr)
    
    def __str__(self):
        return f'r0: {self.r0}\nr1: {self.r1}\nr2: {self.r2}\nr3: {self.r3}\n' + \
               f'r4: {self.r4}\nr5: {self.r5}\nr6: {self.r6}\nr7: {self.r7}'

    def __repr__(self):
        return str(self)
        

In [25]:
ram = RAM8()

In [33]:
ram.eval(inp=(1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0), load=0, addr=(0,1,0))

(1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [34]:
ram

r0: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
r1: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
r2: 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
r3: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
r4: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
r5: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
r6: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
r7: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0